# 方差齐性检验方法
**方差齐性（Homoscedasticity）**指不同组别的数据方差相等，是许多统计方法（如t检验、ANOVA、线性回归）的前提假设。若方差不齐（Heteroscedasticity），可能导致结果偏差。

## Levene检验

### 基本原理
**Levene检验**是一种用于检验多组数据方差是否相等（方差齐性）的非参数方法，其核心思想是通过比较各组中数据与各组中心位置（通常是均值、中位数或截尾均值）的偏差的大小是否有显著差异来检验方差是否相等。
- 如果各组的方差相等，那么它们偏离中心位置的平均偏差应该也差不多。
- 如果某一组的变异性更大，它偏离中心的位置的程度也会更大。

### 适用场景
- 连续型数据的方差齐性检验，作为参数检验的前提检查

### 零假设与备择假设
- 零假设（$H_0$）：各组的方差相等（方差齐性）
- 备择假设（$H_1$）：至少有两组的方差不相等（方差不齐）

### 检验统计量&检验步骤
设有$k$个组，每组有$n_i$个观测值，总样本量为$N = \sum_{i=1}^k n_i$。
- **选择中心位置指标**：
    - Levene 方法使用均值
    - Brown–Forsythe 使用 中位数（median）
    - 记每组的中心位置为$\tilde{Y}_i$。 
- **计算每个观测值与其组内中心位置的偏离的绝对值**：
$$Z_{ij} = | Y_{ij} - \tilde{Y}_i |$$
- **对所有$Z_{ij}$进行单因素方差分析（ANOVA）**：
    - 检验不同组的$Z_{ij}$是否显著不同。 
    - 检验统计量为：
$$W = \frac{(N - k)}{(k - 1)} \cdot \frac{\sum_{i=1}^k n_i (\bar{Z}_i - \bar{Z})^2}{\sum_{i=1}^k \sum_{j=1}^{n_i} (Z_{ij} - \bar{Z}_i)^2}$$
    其中：
        - $\bar{Z}_i$：第$i$组的平均偏差 
        - $\bar{Z}$：所有组的平均偏差 
- $W$统计量近似服从$F $分布，自由度为$k - 1$与$N - k$。 

### Levene检验实际案例

#### 案例背景
某电商平台进行AB测试，比较新旧推荐算法对用户页面停留时间（秒）的影响：
- A组（旧算法）：500名用户
- B组（新算法）：500名用户
- 问题：需要检验两组用户的停留时间方差是否相等

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import levene, ttest_ind

# 设置随机种子
np.random.seed(42)

# 模拟数据：A组（方差小），B组（方差大）
group_A = np.random.normal(loc=50, scale=10, size=500)  # 均值50，标准差10
group_B = np.random.normal(loc=50, scale=20, size=500)  # 均值50，标准差20

# 合并为DataFrame
df = pd.DataFrame({
    'Stay_Time': np.concatenate([group_A, group_B]),
    'Group': ['A'] * 500 + ['B'] * 500
})

df

,Stay_Time,Group
0,54.967142,A
1,48.617357,A
2,56.476885,A
3,65.230299,A
4,47.658466,A
...,...,...
995,44.377994,B
996,85.953731,B
997,62.816857,B
998,38.576420,B


#### 确定假设
- 零假设（$H_0$）：两组用户的停留时间方差相等
- 备择假设（$H_1$）：两组用户的停留时间方差不相等

#### `stats.levene()`
- 输入参数
   - `samples`: 要比较的两个或多个样本数组
   - `center`: 用于计算偏差的中心指标
        - `mean`：原始 Levene 方法
        - `median`：Brown–Forsythe 检验
        - `trimmed`：截尾均值
   - `proportiontocut`: 如果 center=`trimmed`，表示截尾比例（如0.05表示两边各截掉5%）
   - `axis`: 数据沿哪个轴比较（默认 0，即按列比较）
   - `nan_policy`: 如何处理 NaN 值：
        - `propagate`：结果为 NaN
        - `omit`：忽略 NaN
        - `raise`：抛出错误
- 返回值
    - `statistic`: Levene 检验的 F 统计量
    - `p`: p值

#### `stats.levene`代码实现

In [12]:
from scipy.stats import levene

# Levene检验（均值离差）
stat, p = levene(group_A, group_B, center='mean')

print(f"Levene检验统计量: {stat:.3f}, p值: {p:.4f}")

# 根据p值判断方差齐性
alpha = 0.05
if p > alpha:
    print("结论：不能拒绝原假设，认为两组方差相等（满足方差齐性）")
else:
    print("结论：拒绝原假设，认为两组方差不相等（不满足方差齐性）")

Levene检验统计量: 173.866, p值: 0.0000
结论：拒绝原假设，认为两组方差不相等（不满足方差齐性）


## Brown–Forsythe 检验

### 基本原理
**Brown-Forsythe检验**是一种用于检验多组数据方差齐性（Homogeneity of Variance）的非参数方法，由Morton Brown和Alan Forsythe于1974年提出。它是Levene检验的改进版本，通过使用中位数离差替代均值离差，对非正态数据和异常值具有更强的鲁棒性。
- Brown-Forsythe检验的核心思想是通过比较各组数据中位数绝对离差（Median Absolute Deviation, MAD）的差异，判断方差是否相等。

### 零假设与备择假设
- 零假设（$H_0$）：各组的方差相等（方差齐性）
- 备择假设（$H_1$）：至少有两组的方差不相等（方差不齐）

### 检验统计量&检验步骤
设有$k$个组，每组有$n_i$个观测值，总样本量为$N = \sum_{i=1}^k n_i$。
- **选择中心位置指标**：
    - Levene 方法使用均值
    - Brown–Forsythe 使用 中位数（median）
    - 记每组的中心位置为$\tilde{Y}_i$。 
- **计算每个观测值与其组内中心位置的偏离的绝对值**：
$$Z_{ij} = | Y_{ij} - \tilde{Y}_i |$$
- **对所有$Z_{ij}$进行单因素方差分析（ANOVA）**：
    - 检验不同组的$Z_{ij}$是否显著不同。 
    - 检验统计量为：
$$W = \frac{(N - k)}{(k - 1)} \cdot \frac{\sum_{i=1}^k n_i (\bar{Z}_i - \bar{Z})^2}{\sum_{i=1}^k \sum_{j=1}^{n_i} (Z_{ij} - \bar{Z}_i)^2}$$
    其中：
        - $\bar{Z}_i$：第$i$组的平均偏差 
        - $\bar{Z}$：所有组的平均偏差 
- $W$统计量近似服从$F $分布，自由度为$k - 1$与$N - k$。 

### Brown-Forsythe检验实际案例

#### 案例背景
某电商平台进行AB测试，比较新旧推荐算法对用户页面停留时间（秒）的影响：
- A组（旧算法）：500名用户
- B组（新算法）：500名用户
- 问题：需要检验两组用户的停留时间方差是否相等

#### 确定假设
- 零假设（$H_0$）：两组用户的停留时间方差相等
- 备择假设（$H_1$）：两组用户的停留时间方差不相等

#### `stats.levene`代码实现

In [13]:
from scipy.stats import levene

# Brown-Forsythe检验（中位数离差）
stat, p = levene(group_A, group_B, center='median')

print(f"Brown-Forsythe检验统计量: {stat:.3f}, p值: {p:.4f}")

# 根据p值判断方差齐性
alpha = 0.05
if p > alpha:
    print("结论：不能拒绝原假设，认为两组方差相等（满足方差齐性）")
else:
    print("结论：拒绝原假设，认为两组方差不相等（不满足方差齐性）")

Brown-Forsythe检验统计量: 173.837, p值: 0.0000
结论：拒绝原假设，认为两组方差不相等（不满足方差齐性）
